In [1]:
import selfies as sf
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity
import os
import sys
import numpy as np
from numba import jit,njit,prange,vectorize,int64,float64, guvectorize

from functools import partial
import multiprocessing as mp

In [2]:
def selfies_split(selfies):
    return selfies.replace(']', '] ').split()
    
def tokens2int_array(selfies_tokens, vocab_stoi):
    return np.array([vocab_stoi[token] for token in selfies_tokens])

def int_array2tokens(int_array,vocab_itos):
    return [vocab_itos[int_token]for int_token in int_array]

def int_array2selfies(int_array,vocab_itos):
    return "".join(int_array2tokens(int_array,vocab_itos))


def smi2ECFP4(smi):
    ''' compute ECFP4 fingerprint from SMILES 

    Parameters: 
    smi (SMILES) : string  

    Returns: 
    rdkit ECFP4 fingerprint object for mol
    '''
    mol=Chem.MolFromSmiles(smi,sanitize=True)
    return AllChem.GetMorganFingerprint(mol, 2)

def compute_tanimoto(mutated_selfies,fp_target):
    if mutated_selfies:
        smi= sf.decoder(mutated_selfies)
        if smi:
            fp_mol=smi2ECFP4(smi)
            return TanimotoSimilarity(fp_mol,fp_target)
        else: 
            return 0
    else:
        return 0

def tanimoto_scoring(int_array,vocab_itos,fp_target):
    mutated_selfies= int_array2selfies(int_array=int_array,vocab_itos=vocab_itos)
    return compute_tanimoto(mutated_selfies=mutated_selfies,fp_target=fp_target)

@njit
def mutate_selfies(selfies,position,int_token):
    ### replace token 
    selfies[position]=int_token
    return selfies

@njit(parallel=True)
def multiprocess_mutate(selfies_tokens_list,pair_list):
    for i in prange(len(selfies_tokens_list)):
        selfies_tokens_list[i]=mutate_selfies(selfies_tokens_list[i],pair_list[i][0],pair_list[i][1])
    return selfies_tokens_list

@guvectorize([(int64[:], int64[:], int64[:])], '(n),(n)->(n)')
def compute_identity(mutated_selfies_list,target_tokens_array,results):
    for i in range(mutated_selfies_list.shape[0]):
        if mutated_selfies_list[i]== target_tokens_array[i]:
            results[i]=1
        else:
            results[i]=0

# @njit
def homology_scoring(identity_array,target_length):
    sum_array=np.sum(identity_array,axis=1)
    return sum_array/target_length

@njit
def do_sus(values_fitness,nb_childs):
    total_fitness= np.sum(values_fitness)
    pointer_distance=total_fitness/nb_childs
    start= np.random.uniform(0,pointer_distance)
    pointers=np.add(np.repeat(start,nb_childs),np.multiply(np.repeat(pointer_distance,nb_childs),np.arange(np.float64(nb_childs))))
    index_to_keep=np.zeros(nb_childs,dtype=np.int64)
    fitness_cumsum= np.cumsum(values_fitness)
    fitness_index=0
    for i in range(nb_childs):
        while  pointers[i]> fitness_cumsum[fitness_index]:
            fitness_index+=1
        index_to_keep[i]= fitness_index
    return index_to_keep
    
@njit(parallel=True)
def multriprocess_crossover(parent_pop,tokens_length,descendant_pop_size,triplet_list):
    descendant_pop=np.zeros((descendant_pop_size,tokens_length),dtype=np.int64)
    counter=0
    for i in prange(0,descendant_pop.shape[0]):
        if i % 2:
            descendant_pop[i]= np.concatenate((parent_pop[triplet_list[counter,0],:triplet_list[counter,2]],parent_pop[triplet_list[counter,1],triplet_list[counter,2]:]),axis=0)
            descendant_pop[i]= np.concatenate((parent_pop[triplet_list[counter,1],:triplet_list[counter,2]],parent_pop[triplet_list[counter,0],triplet_list[counter,2]:]),axis=0)
            counter+=1
        else:
            pass
    return descendant_pop



In [3]:
start_smiles='CC1=C(C2=C(CCC(O2)(C)COC3=CC=C(C=C3)CC4C(=O)NC(=O)S4)C(=C1O)C)C'
pop_size=1000
parent_pop_size=200000
gen_max=1000
descendant_pop_size= pop_size-parent_pop_size

In [4]:
fp_target=smi2ECFP4(start_smiles)
start_selfies= sf.encoder(start_smiles)
tokens=selfies_split(start_selfies)
counter= 0 
vocab_stoi={}
for token in tokens:
    if token not in vocab_stoi:
        vocab_stoi[token]=counter
        counter+=1
    else: 
        pass

vocab_itos= {integer: token for token, integer in vocab_stoi.items()}
target_tokens_array=tokens2int_array(tokens,vocab_stoi=vocab_stoi)
pop=np.random.randint(len(vocab_stoi),size=(pop_size,len(tokens)))

In [5]:
gen_counter=0

while gen_counter < gen_max:

    ### begin time counter
    start= time.perf_counter()

    ### scoring
    identity_array=np.zeros(pop.shape,dtype=np.int64)
    identity_array=compute_identity(pop,target_tokens_array)
    homology_scores=homology_scoring(identity_array,len(target_tokens_array))
    
    ### scoring 
    with mp.Pool(4) as pool:
        tanimoto_scores=np.array(pool.map(partial(tanimoto_scoring,vocab_itos=vocab_itos,fp_target=fp_target),pop))
        pool.close()
        pool.join()
    
    ### check if stop criterion is met
    total_score= homology_scores + 0.2*tanimoto_scores
    max_homology= np.max(homology_scores)
    max_tanimoto= np.max(tanimoto_scores)
    max_score=np.max(total_score)
    if max_score== 1.2:
        break
    else:
        pass
    
    seed= np.array((pop[total_score.argmax()],pop[total_score.argmax()]))
    pop= np.repeat(seed,pop_size/2,axis=0)

    # ### check if stop criterion is met
    # max_score=np.max(homology_scores)
    # if max_score== 1:
    #     break
    # else:
    #     pass
    
    # seed= np.array((pop[homology_scores.argmax()],pop[homology_scores.argmax()]))
    # pop= np.repeat(seed,pop_size/2,axis=0)
    

    # ### selection
    # index_to_keep=do_sus(homology_scores,parent_pop_size)
    # parent_pop= pop[index_to_keep]

    # ### crossover
    # triplet_list=np.random.randint(0,[parent_pop_size,parent_pop_size,len(tokens)-1],size=(descendant_pop_size//2,3))
    # descendant_pop=multriprocess_crossover(parent_pop,len(tokens),descendant_pop_size,triplet_list)
    # pop= np.concatenate((parent_pop,descendant_pop),axis=0)

    ### mutation
    pair_list=np.random.randint(0,[len(tokens),len(vocab_itos)],size=(pop_size,2))
    pop=multiprocess_mutate(pop,pair_list)

    ### print statement 
    duration= time.perf_counter()-start
    print(f'generation n°{gen_counter} it took {duration:.2f} sec and max score is {max_score} with max tanimoto: {max_tanimoto} and max homology: {max_homology}')

    ### increment generation
    gen_counter+=1



generation n°0 it took 0.65 sec and max score is 0.21477056747216944 with max tanimoto: 0.12 and max homology: 0.20689655172413793
generation n°1 it took 0.22 sec and max score is 0.24152923538230886 with max tanimoto: 0.1016949152542373 and max homology: 0.22413793103448276
generation n°2 it took 0.23 sec and max score is 0.26067755595886266 with max tanimoto: 0.11926605504587157 and max homology: 0.2413793103448276
generation n°3 it took 0.23 sec and max score is 0.2821501014198783 with max tanimoto: 0.11965811965811966 and max homology: 0.25862068965517243
generation n°4 it took 0.24 sec and max score is 0.300055617352614 with max tanimoto: 0.13934426229508196 and max homology: 0.27586206896551724
generation n°5 it took 0.26 sec and max score is 0.3172969966629588 with max tanimoto: 0.14516129032258066 and max homology: 0.29310344827586204
generation n°6 it took 0.22 sec and max score is 0.3359448275862069 with max tanimoto: 0.14516129032258066 and max homology: 0.3103448275862069
g

In [7]:
mutated_selfies_list=

In [9]:
with mp.Pool(4) as pool:
    tanimoto_scores=np.array(pool.map(partial(tanimoto_scoring,vocab_itos=vocab_itos,fp_target=fp_target),mutated_selfies_list))
    pool.close()
    pool.join()

In [41]:
len(set(tanimoto_scores))

1023

In [9]:
# @njit(nogil=True)
# def levenshtein_dist(target_tokens_array,mutated_selfies):
#     """ Levenshtein distance
#           using Dynamic-Programming strategy
#     Parameters
#     ----------
#     target_tokens_array,mutated_selfies : np.array of integers
#     Returns
#     -------
#     int : distance
#     np.array : distance matrix
#     """
#     lev_matrix=np.zeros((len(target_tokens_array), len(target_tokens_array)),dtype=np.int64)
#     for i in range(len(target_tokens_array)):
#         for j in range(len(mutated_selfies)):
#             # we did this before (for when i or j are 0)
#             if min([i, j]) == 0:
#                 lev_matrix[i, j] = max([i, j])
#             else:
#                 # calculate our three possible operations
#                 x = lev_matrix[i-1, j]  # deletion
#                 y = lev_matrix[i, j-1]  # insertion
#                 z = lev_matrix[i-1, j-1]  # substitution
#                 # take the minimum (eg best path/operation)
#                 lev_matrix[i, j] = min([x, y, z])
#                 # and if our two current characters don't match, add 1
#                 if target_tokens_array[i] != mutated_selfies[j]:
#                     # if we have a match, don't add 1
#                     lev_matrix[i, j] += 1
#     return lev_matrix[-1,-1]

# @njit(nogil=True)
# def fitness_scoring(target_selfies,mutated_selfies,):
#     return 1/(1+levenshtein_dist(target_selfies,mutated_selfies))

# @njit(parallel=True)
# def multiprocess_fitness_scoring(target_tokens_array,mutated_selfies_list):
#     size=mutated_selfies_list.shape
#     results=np.zeros(size[0],dtype=np.float64)
#     for i in prange(len(mutated_selfies_list)):
#         results[i]=fitness_scoring(target_tokens_array,mutated_selfies_list[i])
#     return results

# # @njit
# def wrapper_fitness_scoring(target_tokens_array,mutated_selfies_list):
#     size=mutated_selfies_list.shape
#     results=np.zeros(size[0],dtype=np.float64)
#     for i in range(len(mutated_selfies_list)):
#         results[i]=fitness_scoring(target_tokens_array,mutated_selfies_list[i])
#     return results

In [14]:
# @vectorize([int64(int64,int64)],nopython=True)
# def compute_identity(ref_token,mutated_token):
#     if ref_token==mutated_token:
#         return 1
#     else:
#         return 0
# @njit
# def homology_scoring (target_tokens_array,mutated_selfies):
#     homology_score=0.
#     identity_array=compute_identity(target_tokens_array,mutated_selfies)
#     homology_score= np.sum(identity_array)/len(target_tokens_array)
#     return homology_score

In [60]:


triplet_list.shape

(400000, 3)

In [64]:
parent_pop[0,:12]

array([10,  0,  5,  4,  4,  4,  0, 10, 10,  2,  7, 10])

In [71]:
counter=0
parent_pop[triplet_list[counter,0],:triplet_list[counter,2]]

array([ 7,  7,  2,  1,  6,  8,  1,  6,  4,  3,  7, 10,  6,  0,  5, 10,  8,
        4,  4,  3,  9,  6,  4,  3,  3])

In [80]:
%%timeit


145 ms ± 4.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
%%timeit
descendant_pop.shape

50.7 ns ± 0.0735 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [78]:
new_pop=np.repeat(new_pop,coef,axis=0)

In [ ]:
def()